# Построение и оптимизация модели

В этом задании вам предстоит поучаствовать в соревновании на [kaggle inclass](https://inclass.kaggle.com/c/telecom-clients-churn-prediction).
В соревновании вы будете работать с той же выборкой, что и ранее, поэтому воспользуйтесь результатами, полученными на предыдущих неделях. Для успешного участия в соревновании необходимо преодолеть по качеству beseline решение.

Итак, мы научились обрабатывать данные, выбрали схему кросс-валидации и определились с метриками качества. Пора переходить к оптимизации модели. На этой неделе вам предстоит принять участие в соревновании на платформе kaggle inclass! Цель такого соревнования - преодолеть предложенное baseline решение, а, главное, обсудить и сравнить предложенные решения на форуме. Какие признаки оказали наибольший вклад в модель? Как лучше обрабатывать категориальные признаки? Нужно ли делать отбор признаков, А балансировать выборку? Экспериментируйте с данными и обсуждайте ваши решения на форуме!

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score, log_loss

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

seed = 9202
first_categorial_index = 190

Загрузим train dataset и приведем его к стандарному для проекта виду.

In [2]:
# загрузим данные для обучения модели
churn_data_frame = pd.read_csv("orange_small_churn_train_data.csv", ",")
#удалим строки для которых отсутствует значение в labels
churn_data_frame = churn_data_frame.dropna(axis='index', subset=['labels'])
# приведем данные в привычный для проекта вид
churn_labels_frame = churn_data_frame.drop(churn_data_frame.columns[0:231], axis='columns')
churn_data_frame = churn_data_frame.drop(churn_data_frame.columns[0], axis='columns')
churn_data_frame = churn_data_frame.drop(churn_data_frame.columns[-1], axis='columns')

print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(18298, 230)
(18298, 1)


## Общая предобработка признаков

In [3]:
def fill_numericna(train_frame, test_frame, averageCalculator):
    """ Функция заполняет значения в числовом фрейме значениями, посчитанными averageCalculator. """
    
    # Посчитаем средние по колонкам
    numeric_avgs = averageCalculator(train_frame)
    
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_avgs = numeric_avgs.dropna()
    dropped_columns = train_frame.columns.drop(numeric_avgs.index)
    n_frame_train = train_frame[list(numeric_avgs.index)]
    n_frame_test = test_frame[list(numeric_avgs.index)]
    
    # Заполним пропущенные численные значения средними
    n_frame_train = n_frame_train.fillna(numeric_avgs, axis=0)
    n_frame_test = n_frame_test.fillna(numeric_avgs, axis=0)
    return (n_frame_train, n_frame_test, dropped_columns)

def fill_numericna_means(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме средними и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.mean(axis=0, skipna=True))

def fill_numericna_medians(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме медианами и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.median(axis=0, skipna=True))
    
def remove_constant_features(frame, min_count=2):
    """Функция удаляет колонки, которые содержат только одно значение."""
    
    # Посчитаем количества уникальных значений по колонкам
    unique_counts = frame.nunique()
    # Удалим колонки с количеством значений меньшим min_count
    columns_to_drop = unique_counts[unique_counts < min_count].index
    
    return (frame.drop(columns=columns_to_drop), columns_to_drop)

def fill_na_frequent_values(frame):
    """ Функция заполняет пустые значения самым частым значением в колонке. """
    result = frame.copy()
    for column in result.columns:
        frequencies = result[column].value_counts()
        if (len(frequencies) < 1):
            continue
        most_frequent_value = frequencies.index[0]
        result[column] = result[column].fillna(most_frequent_value)
    return result

class MatrixLabelEncoder:
    """ Класс кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    
    def __init__(self):
        self.encoders = []
    
    def fit(self, matrix):
        for column_number in range(matrix.shape[1]):
            column = matrix[:,column_number]
            labelEncoder = LabelEncoder().fit(column)
            self.encoders.append(labelEncoder)
        return self
    
    def transform(self, matrix):
        transformed = np.empty(matrix.shape)
        for column_number in range(matrix.shape[1]):
            labelEncoder = self.encoders[column_number]
            num_column = labelEncoder.transform(matrix[:,column_number])
            for row_number, val in enumerate(num_column):
                transformed[row_number, column_number] = val
        return transformed
    
class CompositeEncoder:
    def __init__(self, encoder_factories):
        self.encoder_factories = encoder_factories
        
    def fit(self, matrix):
        encoders = []
        transformed = matrix
        for encoder_factory in self.encoder_factories:
            encoder = encoder_factory().fit(transformed)
            encoders.append(encoder)
            transformed = encoder.transform(transformed)
        self.encoders = encoders
        return self

    def transform(self, matrix):
        for encoder in self.encoders:
            matrix = encoder.transform(matrix)
        return matrix

Выделим числовые и категориальные признаки.

In [4]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

In [5]:
def predict_ridge_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn модель Ridge """
    
    func = model.decision_function(X)
    return np.exp(func) / (1 + np.exp(func))

def predict_model_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn """
    return list(zip(*model.predict_proba(X)))[1]

def get_model_data(model_with_data):
    """ Функция принимает на вход результат stratifiedKFold_fscore и возвращает только модель и её X_test и y_test """
    model = model_with_data[1]
    split = model_with_data[3]
    X = split[2]
    y = split[3]
    return (model,X,y)

def calculate_metrics(model_with_data, predict_probabilities):
    """ Посчитаем метрики качества для модели """
    model, X, y = get_model_data(model_with_data)
    predictions = model.predict(X)
    probabilities = predict_probabilities(X, model)
    # Считаем F-меру, precision и recall
    fscore = f1_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    # Считаем Log loss
    logLoss = log_loss(y, probabilities)
    # Считаем roc auc score
    rocAuc = roc_auc_score(y, probabilities)
    return (precision,recall,fscore,logLoss,rocAuc)

def transform_to_chart_model(label, model_with_data, predict_probabilities):
    """ Функция принимает на вход заголовок, результат функции stratifiedKFold_fscore и функцию, считающую вероятности
    и возвращает заголовок, вектор ответов и вектор вероятностей для построения графиков """
    model, X, y = get_model_data(model_with_data)
    probabilities = predict_probabilities(X, model)
    return (label, y, probabilities)

def charts_row(model_charts, chart_builder, figsize=(11, 4)):
    """ Функция принимает данные для построения графиков и логику построения одного графика и строит графики в одну строку """
    fig, axes = plt.subplots(
        nrows=1,
        ncols=len(model_charts),
        figsize=figsize,
        sharey=True)
    for i, chart_data in enumerate(model_charts):
        ax = axes[i]
        chart_builder(chart_data, ax, i)
    plt.show()
    
def scatter(chart_data, ax, ax_index, T=0.5):
    """ Функция строит график распределения вероятностей по классам """
    label, actual, predicted = chart_data
    ax.scatter(actual, predicted)
    ax.set_xlabel("Labels")
    if ax_index == 0:
        ax.set_ylabel("Predicted probabilities")
    ax.set_title(label)
    ax.plot([-1.1, 1.2], [T, T])
    ax.axis([-1.1, 1.1, -0.1, 1.1])
    
def precision_recal_thresh(chart_data, ax, ax_index):
    """ Функция строит графики значений precision и recall в зависимости от порога """
    label, actual, predicted = chart_data
    prec, rec, thresh = precision_recall_curve(actual, predicted)
    min_len = min([len(prec),len(rec),len(thresh)])
    ax.plot(thresh[:min_len], prec[:min_len], label="precision")
    ax.plot(thresh[:min_len], rec[:min_len], label="recall")
    ax.legend(loc=1)
    ax.set_xlabel("threshold")
    ax.set_title(label)
    
def auc_prc(chart_data, ax, ax_index):
    """ Функция строит график AUC PRC (зависимость precision от recall)"""
    label, actual, predicted = chart_data
    prec, rec, thresh = precision_recall_curve(actual, predicted)
    min_len = min([len(prec),len(rec),len(thresh)])
    ax.plot(rec[:min_len], prec[:min_len])
    ax.set_xlabel("recall")
    if ax_index == 0:
        ax.set_ylabel("precision")
    ax.set_title(label)

def calc_min_distance (actual,predicted):
    """ Функция считает минимальное расстояние до точки [0;1] для AUC ROC """
    fpr,tpr,thr = roc_curve(actual,predicted)
    distance,fpr_v,tpr_v,thr_v = min(zip(np.sqrt((1.-tpr)**2+fpr**2),fpr,tpr,thr), key=lambda d:d[0])
    return (distance,fpr_v,tpr_v,thr_v)
    
def auc_roc(chart_data, ax, ax_index):
    """ Функция строит кривую AUC ROC и отмечает точку, соответствующую минимальному расстоянию до точки [0;1] """
    label, actual, predicted = chart_data
    fpr, tpr, thr = roc_curve(actual, predicted)
    min_dist,min_fpr,min_tpr,_ = calc_min_distance(actual, predicted)
    ax.plot(fpr, tpr, label="ROC AUC curve")
    ax.scatter(min_fpr,min_tpr,color="red")
    ax.set_xlabel("false positive rate")
    if ax_index == 0:
        ax.set_ylabel("true positive rate")
    ax.legend(loc=4)
    ax.set_title(label)

In [6]:
def stratifiedKFold_fscore(
    frame,
    labels,
    model_factory,
    process_frame,
    frame_to_matrix,
    numeric_features,
    categorial_features,
    predict_probabilities,
    seed,
    folds_count = 5):
    """ Функция разбивает набор данных на folds_count, считает ROC-AUC на каждом фолде
        и возвращает усредненное по фолдам значение. Лучше всего на прошлой недели работало разбиение на 5 фолдов.
        Функция также возвращает модель, показавшую лучшее качество, её метрики и разделение данных.
        Разделение данных нужно для того, чтобы строить метрики модели на данных, на которых она не обучалась."""
    skf = StratifiedKFold(n_splits=folds_count, shuffle=True, random_state=seed)
    
    best_model = None
    best_score = 0
    best_precision = 0
    best_recall = 0
    best_table = None
    best_split = None
    best_encoders = None
    best_dropped_columns = None
    metrics_sum = 0
    for train_indices, test_indices in skf.split(frame, labels):
        # Разобьем фрейм на train и test с помощью функции process_frame
        # Внутри такой функции мы можем по-разному обрабатывать признаки обучаясь только на train наборе.
        train_frame, train_labels, test_frame, test_labels, dropped_numeric, dropped_categorial = process_frame(
            frame.loc[train_indices, :],
            labels.loc[train_indices, :],
            frame.loc[test_indices, :],
            labels.loc[test_indices, :],
            numeric_features,
            categorial_features)
        numeric_cleaned = numeric_features.drop(dropped_numeric)
        categorial_cleaned = categorial_features.drop(dropped_categorial)
        # Преобразуем фреймы в матрицы.
        # Тут можно выполнить финальное преобразование признаков, например масштабирование признаков.
        # В функции frame_to_matrix энкодеры типа StandardScaler обучаются только на train признаках.
        X_train, X_test, num_encoder, cat_encoder = frame_to_matrix(
            train_frame,
            test_frame,
            numeric_cleaned,
            categorial_cleaned)
        y_train = train_labels.as_matrix().flatten()
        y_test = test_labels.as_matrix().flatten()

        model = model_factory()
        # Обучим модель
        model.fit(X_train, y_train)
        
        # Построим вероятности принадлежности к целевому классу
        probabilities = predict_probabilities(X_test, model)
        # Считаем roc auc score
        rocAuc = roc_auc_score(y_test, probabilities)
        metrics_sum += rocAuc
        if(best_model is None or best_score < rocAuc):
            # В случае, если модель лучше предыдущих сохраним её
            # оценку, модель, матрицу ошибок и разделение данных
            best_score = rocAuc
            best_model = model
            best_dropped_columns = (dropped_numeric, dropped_categorial)
            best_encoders = (num_encoder, cat_encoder)
            best_split = (X_train, y_train, X_test, y_test)
    return (
        metrics_sum/folds_count,
        best_model,
        best_score,
        best_split,
        best_encoders,
        best_dropped_columns)

In [7]:
def cleanup_frame_common(frame, numeric_features, categorial_features):
    """Функция делит признакина числовые и категориальные и удаляет константные признаки, содержащие только одно значение"""
    # Разделим коллекции на группы - числовые и категориальные.
    numeric_frame = frame[numeric_features].copy()
    categorial_frame = frame[categorial_features].copy()
    # Удалим вещественные колонки, содержащие одно и менее значений. 0 значений мы получаем, когда значения во всех строках Nan.
    numeric_frame_no_const, dropped_const_numeric_columns = remove_constant_features(numeric_frame)
    
    # Удалим категориальные колонки, содержащие ноль значений. Если есть одно значение, то могут быть Nan, которые для
    # категориальных признаков могут быть еще одной категорией (зависит от стратегии обработки).
    categorial_frame_no_const, dropped_const_categorial_columns = remove_constant_features(categorial_frame, 1)
    
    # Восстановим фрейм и вернем вместе с ним список удаленных категориальных колонок.
    return (pd.concat([numeric_frame, categorial_frame], axis=1),
            list(dropped_const_numeric_columns),
            list(dropped_const_categorial_columns))

def process_frame_base_model(train_frame, train_labels, test_frame, test_labels, numeric_features, categorial_features):
    """ Функция строит базовую модель из предыдущей недели """
    
    # Удалим константные колонки из train_frame, и такие-же колонки из test_frame
    train_frame, const_numeric_columns, const_categorial_columns = cleanup_frame_common(
        train_frame,
        numeric_features,
        categorial_features)
    test_frame = test_frame.drop(columns=const_numeric_columns)
    test_frame = test_frame.drop(columns=const_categorial_columns)
    
    numeric_features = numeric_features.drop(const_numeric_columns)
    categorial_features = categorial_features.drop(const_categorial_columns)
    
    # Заполним пропущенные вещественные значения средними
    numeric_train, numeric_test, dropped_numeric = fill_numericna_means(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    numeric_features = numeric_features.drop(dropped_numeric)
    
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    categorial_train = train_frame[categorial_features].fillna("NaV")
    categorial_test = test_frame[categorial_features].fillna("NaV")
    
    # Удалим категориальные колонки с одним единственным значением
    categorial_train, dropped_categorial = remove_constant_features(categorial_train)
    categorial_test = categorial_test.drop(columns=dropped_categorial)
    
    categorial_features = categorial_features.drop(dropped_categorial)
    
    # Список удаленных колонок
    dropped_numeric = np.concatenate([
        list(const_numeric_columns),
        list(dropped_numeric)])
    dropped_categorial = np.concatenate([
        list(const_categorial_columns),
        list(dropped_categorial)])
    
    return (pd.concat([numeric_train, categorial_train], axis=1),
            train_labels,
            pd.concat([numeric_test, categorial_test], axis=1),
            test_labels,
            dropped_numeric,
            dropped_categorial)

def scale_features(train_frame, test_frame):
    train_numeric = train_frame.as_matrix()
    
    scaler = StandardScaler().fit(train_numeric)
    
    train_numeric = coo_matrix(scaler.transform(train_numeric))
    test_numeric = coo_matrix(scaler.transform(test_frame.as_matrix()))
    
    return (train_numeric, test_numeric, scaler)

def one_hot_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    
    if fit_matrix.shape[0] == 0 or fit_matrix.shape[1] == 0:
        return (coo_matrix(train_frame.as_matrix()), coo_matrix(test_frame.as_matrix()), None)
    categorial_encoder = CompositeEncoder([MatrixLabelEncoder, OneHotEncoder]).fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def int_label_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    categorial_encoder = MatrixLabelEncoder().fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def frame_to_matrix_one_hot(train_frame, test_frame, numeric_features, categorial_features):
    """ Функци преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные с помощью OneHotEncoding. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def frame_to_matrix_labeled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def ridge_one_hot_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        process_frame_base_model,
        frame_to_matrix_one_hot, 
        numeric_columns,
        categorial_columns,
        predict_ridge_proba,
        seed)

def random_forest_labeled_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

def gradient_boosting_labeled_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

Базовые модели, построенные на предыдущей неделе

In [8]:
ridge_one_hot_base = ridge_one_hot_builder(churn_data_frame, churn_labels_frame)
random_forest_base = random_forest_labeled_builder(churn_data_frame, churn_labels_frame)
gradient_boosting_base = gradient_boosting_labeled_builder(churn_data_frame, churn_labels_frame)

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use 

## Sampling
Проверим увеличит ли сэмплирование данных качество модели

In [9]:
def balance_frame_sampling(frame_to_balance, labels, repeat_times):
    """ Функция балансирует соотношения классов сэмплируя класс churn. """
    y_name = "labels"
    churn_only_labels = labels[labels[y_name] == 1]
    churn_indices = list(churn_only_labels.index)
    churn_only_frame = frame_to_balance.loc[churn_indices,:]
    churn_balanced_frame = frame_to_balance.copy()
    churn_balanced_labels = labels.copy()
    for i in range(0, (repeat_times-1)):
        churn_balanced_frame = pd.concat([churn_balanced_frame, churn_only_frame], ignore_index=True)
        churn_balanced_labels = pd.concat([churn_balanced_labels, churn_only_labels], ignore_index=True)
        
    churn_balanced_frame[y_name] = churn_balanced_labels[y_name]
    churn_balanced_frame = churn_balanced_frame.sample(frac=1).reset_index(drop=True)
    
    churn_balanced_labels = pd.DataFrame(churn_balanced_frame[y_name], columns=[y_name])
    churn_balanced_frame = churn_balanced_frame.drop(columns=[y_name])
    
    return (churn_balanced_frame, churn_balanced_labels)

def balance_train_frame(train_frame, train_labels, test_frame, test_labels, numeric_features, categorial_features, repeat_times):
    """ Функция балансирует train frame, после этого обрабатывает признаки с помощью метода
        process_frame_base_model """
    balanced_train, balanced_labels = balance_frame_sampling(train_frame, train_labels, repeat_times)
    return process_frame_base_model(balanced_train, balanced_labels, test_frame, test_labels, numeric_features, categorial_features)

def balanced_ridge_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RidgeClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_one_hot,
        numeric_features,
        categorial_features,
        predict_ridge_proba,
        seed)

def balanced_random_forest_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RandomForestClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

def balanced_gradient_boosting_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе GradientBoostingClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

Построим RidgeClassifier, RandomForestClassifier и GradientBoostingClassifier и сравним результат с базовым решением

In [10]:
repeat_times = 12 # известно соотношение примерно 1 к 12 для оттока
ridge_balanced = balanced_ridge_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)
random_forest_balanced = balanced_random_forest_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)
gradient_boosting_balanced = balanced_gradient_boosting_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

In [11]:
indices = ["Ridge Base", "Ridge Balanced", "Random Forest Base", "Random Forest Balanced", "Gradient boosting base", "Gradient boosting balanced"]
print(pd.DataFrame(
    [
        [ridge_one_hot_base[0], ridge_one_hot_base[2]],
        [ridge_balanced[0], ridge_balanced[2]],
        [random_forest_base[0], random_forest_base[2]],
        [random_forest_balanced[0], random_forest_balanced[2]],
        [gradient_boosting_base[0], gradient_boosting_base[2]],
        [gradient_boosting_balanced[0], gradient_boosting_balanced[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        calculate_metrics(ridge_one_hot_base, predict_ridge_proba),
        calculate_metrics(ridge_balanced, predict_ridge_proba),
        calculate_metrics(random_forest_base, predict_model_proba),
        calculate_metrics(random_forest_balanced, predict_model_proba),
        calculate_metrics(gradient_boosting_base, predict_model_proba),
        calculate_metrics(gradient_boosting_balanced, predict_model_proba),
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                                Mean      Best
Ridge Base                  0.684714  0.689121
Ridge Balanced              0.684916  0.687856
Random Forest Base          0.604407  0.619388
Random Forest Balanced      0.618826  0.638221
Gradient boosting base      0.735268  0.740807
Gradient boosting balanced  0.731612  0.740321
()


C:\Users\Pavel\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Pavel\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


                            Precision    Recall   F-Score  Log Loss   Roc Auc
Ridge Base                   0.000000  0.000000  0.000000  0.414818  0.689121
Ridge Balanced               0.130370  0.591597  0.213657  0.629100  0.687856
Random Forest Base           0.375000  0.005042  0.009950  0.946863  0.619388
Random Forest Balanced       0.157895  0.005042  0.009772  0.776416  0.638221
Gradient boosting base       0.750000  0.015126  0.029654  0.236170  0.740807
Gradient boosting balanced   0.151789  0.648739  0.246017  0.564453  0.740321


Из таблицы выше видно, что при балансировке классов стабильно растет Recall и F-Score. Roc Auc при этом чуть-чуть просел аж для двух моделей (Ridge и Gradient Boosting).
Log Loss для Gradient Boosting при балансироваке также вырос.
Судя по тому, что по большей части метрики растут и растут сильно (в десятках процентов) я в данном ноутбуке буду балансировать классы.

## Масштабировать или нет

Попробуем не масшатабировать модель. Сравним результаты и решим, что лучше, - масштабировать признаки или нет.

In [12]:
def to_matrix_one_hot_not_scaled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функци преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные с помощью OneHotEncoding. """
    
    train_numeric = coo_matrix(train_frame[numeric_features].as_matrix())
    test_numeric = coo_matrix(test_frame[numeric_features].as_matrix())
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            None,
            categorial_encoder)

def to_matrix_labeled_not_scaled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    train_numeric = coo_matrix(train_frame[numeric_features].as_matrix())
    test_numeric = coo_matrix(test_frame[numeric_features].as_matrix())
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            None,
            categorial_encoder)

In [13]:
def not_scaled_bal_ridge_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_one_hot_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_ridge_proba,
        seed)

def not_scaled_bal_random_forest_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_labeled_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

def not_scaled_bal_gradient_boosting_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_labeled_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

In [26]:
ridge_not_scaled = not_scaled_bal_ridge_builder(churn_data_frame, churn_labels_frame)
random_forest_not_scaled = not_scaled_bal_random_forest_builder(churn_data_frame, churn_labels_frame)
gradient_boosting_not_scaled = not_scaled_bal_gradient_boosting_builder(churn_data_frame, churn_labels_frame)

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site

In [27]:
indices = ["Ridge Balanced", "Ridge Not Scaled", "Random Forest Balanced",
           "Random Forest Not Scaled", "Gradient boosting balanced", "Gradient boosting Not Scaled"]
print(pd.DataFrame(
    [
        [ridge_balanced[0], ridge_balanced[2]],
        [ridge_not_scaled[0], ridge_not_scaled[2]],
        [random_forest_balanced[0], random_forest_balanced[2]],
        [random_forest_not_scaled[0], random_forest_not_scaled[2]],
        [gradient_boosting_balanced[0], gradient_boosting_balanced[2]],
        [gradient_boosting_not_scaled[0], gradient_boosting_not_scaled[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        calculate_metrics(ridge_balanced, predict_ridge_proba),
        calculate_metrics(ridge_not_scaled, predict_ridge_proba),
        calculate_metrics(random_forest_balanced, predict_model_proba),
        calculate_metrics(random_forest_not_scaled, predict_model_proba),
        calculate_metrics(gradient_boosting_balanced, predict_model_proba),
        calculate_metrics(gradient_boosting_not_scaled, predict_model_proba),
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                                  Mean      Best
Ridge Balanced                0.684918  0.687860
Ridge Not Scaled              0.571251  0.575156
Random Forest Balanced        0.620342  0.640097
Random Forest Not Scaled      0.627024  0.648688
Gradient boosting balanced    0.731634  0.740386
Gradient boosting Not Scaled  0.731630  0.740366
()
                              Precision    Recall   F-Score  Log Loss  \
Ridge Balanced                 0.130419  0.591597  0.213722  0.629100   
Ridge Not Scaled               0.092129  0.507563  0.155951  0.679355   
Random Forest Balanced         0.192308  0.008403  0.016103  0.787784   
Random Forest Not Scaled       0.305556  0.018487  0.034865  0.716556   
Gradient boosting balanced     0.151789  0.648739  0.246017  0.564442   
Gradient boosting Not Scaled   0.151849  0.648739  0.246095  0.564423   

                               Roc Auc  
Ridge Balanced                0.687860  
Ridge Not Scaled              0.575156  
Random Forest Balan

В основном метрики либо не растут, либо уменьшаются. На мой взгляд лучше масштабировать признаки

## Отбор признаков
Строим GradientBoostingClassifier, добавляя по одному вещественные признаки, которые повышают качество.(считал больше часа)

In [36]:
repeat_times=12
num_cols_selected = pd.Index([])
best_metric = 0
best_model = None
best_numeric_features = pd.Index([])

for col in numeric_columns:
    num_cols_selected = num_cols_selected.insert(len(num_cols_selected), col)
    print (num_cols_selected)
    gb_selected_num = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        num_cols_selected,
        pd.Index([u'Var191']),
        repeat_times)
    
    auc_roc = gb_selected_num[2]
    print (auc_roc)
    if np.isin(col, gb_selected_num[5][0], invert=True) and np.round(auc_roc, 5) > np.round(best_metric, 5):
        best_metric = auc_roc
        best_model = gb_selected_num
        best_numeric_features = num_cols_selected.copy()
    else:
        num_cols_selected = num_cols_selected.drop(col)
        
print (best_metric, best_numeric_features)

Index([u'Var1'], dtype='object')


C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

0.5065856025056882
Index([u'Var1', u'Var2'], dtype='object')
0.5139145771760892
Index([u'Var1', u'Var2', u'Var3'], dtype='object')
0.5132182694795909
Index([u'Var1', u'Var2', u'Var4'], dtype='object')
0.5186396881507499
Index([u'Var1', u'Var2', u'Var4', u'Var5'], dtype='object')
0.5233082348401886
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6'], dtype='object')
0.5619363142339594
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7'], dtype='object')
0.598310134426542
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var8'], dtype='object')
0.598310134426542
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9'], dtype='object')
0.5984842397250634
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9',
       u'Var10'],
      dtype='object')
0.5983772115004835
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9',
       u'Var11'],
      dtype='object')
0.5979108499153307
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var

0.6135150562588303
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var52'],
      dtype='object')
0.6149464080027689
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var53'],
      dtype='object')
0.6135211843008641
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54'],
      dtype='object')
0.6175805582192364
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var55'],
      dtype='object')
0.6175846435805923
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9

0.6475480455517791
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83'],
      dtype='object')
0.6487827325393358
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var84'],
      dtype='object')
0.6486750378747044
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var85'],
   

0.6499645367937856
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var106'],
      dtype='object')
0.649709996084862
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var107'],
      dtype='object')
0.6504183523510686
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Va

0.7246936262688736
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var127'],
      dtype='object')
0.7246657096329416
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128'],
      dtype='object')
0.7249520253746333
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13'

0.7239457781762266
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var145'],
      dtype='object')
0.7254632629554185
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var146'],
      dtype='object')
0.7262916834525843
Index([u'Var1', u'Var2', u'Va

0.7263575031633179
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var162', u'Var163'],
      dtype='object')
0.7251205465305636
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var162', u'Var164'],
      dtype='object')
0.7263606806665948
Index([

0.7250248809854799
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var162', u'Var168', u'Var176',
       u'Var180'],
      dtype='object')
0.7264834684717911
Index([u'Var1', u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var162', u'Var168', u'Var176',
       u'

Также по-одному добавим категориальные признаки (долго считает)

In [37]:
cat_cols_selected = pd.Index([])
best_cat_metric = 0
best_cat_model = None
best_cat_features = pd.Index([])

for col in categorial_columns:
    cat_cols_selected = cat_cols_selected.insert(len(cat_cols_selected), col)
    print (cat_cols_selected)
    gb_selected_cat = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        best_numeric_features,
        cat_cols_selected,
        repeat_times)
    
    auc_roc = gb_selected_cat[2]
    print (auc_roc)
    if np.isin(col, gb_selected_cat[5][1], invert=True) and np.round(auc_roc, 5) > np.round(best_cat_metric, 5):
        best_cat_metric = auc_roc
        best_cat_model = gb_selected_cat
        best_cat_features = cat_cols_selected.copy()
    else:
        cat_cols_selected = cat_cols_selected.drop(col)
        
print (best_cat_metric, best_cat_features)

Index([u'Var191'], dtype='object')


C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

0.7382561181123362
Index([u'Var191', u'Var192'], dtype='object')
0.7383937720935775
Index([u'Var191', u'Var192', u'Var193'], dtype='object')
0.7383678981383235
Index([u'Var191', u'Var192', u'Var194'], dtype='object')
0.7382823325143697
Index([u'Var191', u'Var192', u'Var195'], dtype='object')
0.7378197788230755
Index([u'Var191', u'Var192', u'Var196'], dtype='object')
0.7382889144854431
Index([u'Var191', u'Var192', u'Var197'], dtype='object')
0.7365494584059147
Index([u'Var191', u'Var192', u'Var198'], dtype='object')
0.7366383150154052
Index([u'Var191', u'Var192', u'Var199'], dtype='object')
0.7362397653186866
Index([u'Var191', u'Var192', u'Var200'], dtype='object')
0.7361316167250155
Index([u'Var191', u'Var192', u'Var201'], dtype='object')
0.7379661709383281
Index([u'Var191', u'Var192', u'Var202'], dtype='object')
0.7364629849238817
Index([u'Var191', u'Var192', u'Var203'], dtype='object')
0.7380024852614915
Index([u'Var191', u'Var192', u'Var204'], dtype='object')
0.7365896311259144
Inde

Строим GradientBoostingClassifier с отобранными категориальными и вещественными признаками, удаляем те вещественные признаки, которые уменьшают качество.(долго считает)

In [38]:
num_cols_selected = best_numeric_features.copy()
very_best_metric = best_metric
very_best_model = best_model
very_best_num_features = best_numeric_features.copy()

for col in best_numeric_features:
    
    num_cols_selected = num_cols_selected.drop(col)
    print (num_cols_selected)
    gb_selected_num = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        num_cols_selected,
        best_cat_features,
        repeat_times)
    
    auc_roc = gb_selected_num[2]
    print (auc_roc)
    if np.round(auc_roc, 5) >= np.round(very_best_metric, 5):
        very_best_metric = auc_roc
        very_best_model = gb_selected_num
        very_best_num_features = num_cols_selected.copy()
    else:
        num_cols_selected = num_cols_selected.insert(len(num_cols_selected), col)
        
print (very_best_metric, very_best_num_features)

Index([u'Var2', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12',
       u'Var13', u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38',
       u'Var50', u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73',
       u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var123', u'Var126',
       u'Var128', u'Var129', u'Var137', u'Var162', u'Var168', u'Var176',
       u'Var186', u'Var189'],
      dtype='object')


C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

0.745318800038584
Index([u'Var4', u'Var5', u'Var6', u'Var7', u'Var9', u'Var12', u'Var13',
       u'Var17', u'Var18', u'Var28', u'Var35', u'Var36', u'Var38', u'Var50',
       u'Var54', u'Var63', u'Var65', u'Var67', u'Var69', u'Var73', u'Var74',
       u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91', u'Var92',
       u'Var101', u'Var113', u'Var115', u'Var123', u'Var126', u'Var128',
       u'Var129', u'Var137', u'Var162', u'Var168', u'Var176', u'Var186',
       u'Var189'],
      dtype='object')
0.7452686408797145
Index([u'Var5', u'Var6', u'Var7', u'Var9', u'Var12', u'Var13', u'Var17',
       u'Var18', u'Var28', u'Var35', u'Var36', u'Var38', u'Var50', u'Var54',
       u'Var63', u'Var65', u'Var67', u'Var69', u'Var73', u'Var74', u'Var75',
       u'Var78', u'Var81', u'Var83', u'Var87', u'Var91', u'Var92', u'Var101',
       u'Var113', u'Var115', u'Var123', u'Var126', u'Var128', u'Var129',
       u'Var137', u'Var162', u'Var168', u'Var176', u'Var186', u'Var189',
       u'Var2'],
     

0.7463371898387985
Index([u'Var69', u'Var73', u'Var74', u'Var75', u'Var78', u'Var81', u'Var83',
       u'Var87', u'Var91', u'Var92', u'Var101', u'Var113', u'Var115',
       u'Var123', u'Var126', u'Var128', u'Var129', u'Var137', u'Var162',
       u'Var168', u'Var176', u'Var186', u'Var189', u'Var2', u'Var5', u'Var6',
       u'Var7', u'Var17', u'Var35', u'Var36', u'Var38', u'Var54', u'Var63',
       u'Var65'],
      dtype='object')
0.7471905764331391
Index([u'Var73', u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87',
       u'Var91', u'Var92', u'Var101', u'Var113', u'Var115', u'Var123',
       u'Var126', u'Var128', u'Var129', u'Var137', u'Var162', u'Var168',
       u'Var176', u'Var186', u'Var189', u'Var2', u'Var5', u'Var6', u'Var7',
       u'Var17', u'Var35', u'Var36', u'Var38', u'Var54', u'Var63', u'Var65'],
      dtype='object')
0.7455986472914622
Index([u'Var74', u'Var75', u'Var78', u'Var81', u'Var83', u'Var87', u'Var91',
       u'Var92', u'Var101', u'Var113', u'Var115', u'Var

0.7465338545951803
Index([u'Var186', u'Var189', u'Var2', u'Var5', u'Var6', u'Var7', u'Var17',
       u'Var35', u'Var36', u'Var38', u'Var54', u'Var63', u'Var65', u'Var69',
       u'Var73', u'Var74', u'Var81', u'Var83', u'Var91', u'Var92', u'Var101',
       u'Var113', u'Var115', u'Var123', u'Var126', u'Var129', u'Var137',
       u'Var162', u'Var168'],
      dtype='object')
0.7481389476790041
Index([u'Var189', u'Var2', u'Var5', u'Var6', u'Var7', u'Var17', u'Var35',
       u'Var36', u'Var38', u'Var54', u'Var63', u'Var65', u'Var69', u'Var73',
       u'Var74', u'Var81', u'Var83', u'Var91', u'Var92', u'Var101', u'Var113',
       u'Var115', u'Var123', u'Var126', u'Var129', u'Var137', u'Var162',
       u'Var168'],
      dtype='object')
0.7481389476790041
Index([u'Var2', u'Var5', u'Var6', u'Var7', u'Var17', u'Var35', u'Var36',
       u'Var38', u'Var54', u'Var63', u'Var65', u'Var69', u'Var73', u'Var74',
       u'Var81', u'Var83', u'Var91', u'Var92', u'Var101', u'Var113', u'Var115',
       u'Var12

Строим модель, удаляя те категориальные признаки, которые понижают качество (долго считает)

In [40]:
cat_cols_selected = best_cat_features.copy()
very_best_cat_metric = very_best_metric
very_best_cat_model = very_best_model
very_best_cat_features = best_cat_features.copy()
print (cat_cols_selected)
for col in best_cat_features:
    cat_cols_selected = cat_cols_selected.drop(col)
    print (cat_cols_selected)
    gb_selected_cat = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        very_best_num_features,
        cat_cols_selected,
        repeat_times)
    auc_roc = gb_selected_cat[2]
    print (auc_roc)
    if np.round(auc_roc, 5) >= np.round(very_best_cat_metric, 5):
        very_best_cat_metric = auc_roc
        very_best_cat_model = gb_selected_cat
        very_best_cat_features = cat_cols_selected.copy()
    else:
        cat_cols_selected = cat_cols_selected.insert(len(cat_cols_selected), col)
        
print (very_best_cat_metric, very_best_cat_features)

Index([u'Var191', u'Var192', u'Var211', u'Var212', u'Var213', u'Var216',
       u'Var217', u'Var218', u'Var220', u'Var224', u'Var228'],
      dtype='object')
Index([u'Var192', u'Var211', u'Var212', u'Var213', u'Var216', u'Var217',
       u'Var218', u'Var220', u'Var224', u'Var228'],
      dtype='object')


C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

0.7481389476790041
Index([u'Var211', u'Var212', u'Var213', u'Var216', u'Var217', u'Var218',
       u'Var220', u'Var224', u'Var228'],
      dtype='object')
0.746647563819586
Index([u'Var212', u'Var213', u'Var216', u'Var217', u'Var218', u'Var220',
       u'Var224', u'Var228', u'Var192'],
      dtype='object')
0.7481146624753885
Index([u'Var213', u'Var216', u'Var217', u'Var218', u'Var220', u'Var224',
       u'Var228', u'Var192', u'Var211'],
      dtype='object')
0.7433396694261769
Index([u'Var216', u'Var217', u'Var218', u'Var220', u'Var224', u'Var228',
       u'Var192', u'Var211', u'Var212'],
      dtype='object')
0.7481439408984392
Index([u'Var217', u'Var218', u'Var220', u'Var224', u'Var228', u'Var192',
       u'Var211', u'Var212'],
      dtype='object')
0.7458066602738327
Index([u'Var218', u'Var220', u'Var224', u'Var228', u'Var192', u'Var211',
       u'Var212', u'Var216'],
      dtype='object')
0.7465201232417342
Index([u'Var220', u'Var224', u'Var228', u'Var192', u'Var211', u'Var212',
 

In [41]:
calculate_metrics(very_best_cat_model, predict_model_proba)

(0.15049432442328817,
 0.6907563025210084,
 0.24714371617558628,
 0.5733907779385966,
 0.7481389476790041)

In [42]:
all_best_features = pd.Index(list(very_best_num_features) + list(very_best_cat_features))
frame_best_features = churn_data_frame[all_best_features]

Построим ridge, random forest и gradient boosting классификаторы основываясь на отборе признаков проведенном выше

In [43]:
ridge_add_del = balanced_ridge_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
random_forest_add_del = balanced_random_forest_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
gradient_boosting_add_del = balanced_gradient_boosting_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Pavel\Anaconda2\lib\site-packag

In [48]:
indices = ["Ridge add del", "Random forest add del", "GB Add del"]
print(pd.DataFrame(
    [
        
        [ridge_add_del[0], ridge_add_del[2]],
        [random_forest_add_del[0], random_forest_add_del[2]],
        [gradient_boosting_add_del[0], gradient_boosting_add_del[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        
        calculate_metrics(ridge_add_del, predict_ridge_proba),
        calculate_metrics(random_forest_add_del, predict_model_proba),
        calculate_metrics(gradient_boosting_add_del, predict_model_proba)
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                           Mean      Best
Ridge add del          0.662426  0.673215
Random forest add del  0.637682  0.653513
GB Add del             0.731333  0.748145
()
                       Precision    Recall   F-Score  Log Loss   Roc Auc
Ridge add del           0.136975  0.547899  0.219160  0.617831  0.673215
Random forest add del   0.302632  0.038655  0.068554  0.814957  0.653513
GB Add del              0.150494  0.690756  0.247144  0.573385  0.748145


Видно, что на удалось чуть-чуть улучшить модели за счет отбора

## Построение решения для kaggle



In [49]:
kaggle_data_frame = pd.read_csv("orange_small_churn_test_data.csv", ",")

In [50]:
kaggle_data_frame

,ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
5,5,NaN,NaN,NaN,NaN,NaN,140.0,0.0,NaN,NaN,...,oslk,T3zAKIt,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN
6,6,NaN,NaN,NaN,NaN,NaN,6027.0,7.0,NaN,NaN,...,oslk,2JfQ3DB,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
7,7,NaN,NaN,NaN,NaN,NaN,812.0,14.0,NaN,NaN,...,zCkv,catzS2D,jySVZNlOJy,NaN,xG3x,Aoh3,ZI9m,ib5G6X1eUxUn6,mj86,NaN
8,8,NaN,NaN,NaN,NaN,NaN,161.0,0.0,NaN,NaN,...,oslk,QuuQ5_S,LM8l689qOp,NaN,kG3k,7P5s,RAYp,F2FyR07IdsN7I,mj86,NaN
9,9,NaN,NaN,NaN,NaN,NaN,175.0,0.0,NaN,NaN,...,d0EEeJi,MHPNycx,LM8l689qOp,NaN,NaN,me1d,nIGXDli,F2FyR07IdsN7I,NaN,NaN


In [51]:
# Посмотреть насколько больше категорий стало
cols_to_remove = []
print(kaggle_data_frame.shape[0])
for col in very_best_cat_features:
    full_train_col = set(churn_data_frame[col].as_matrix())
    kaggle_col = set(kaggle_data_frame[col].as_matrix())
    intersected = (kaggle_col & full_train_col)
    for v in intersected:
        kaggle_col.remove(v)
    cols_to_remove.append((col, len(kaggle_col)))
print (list(ctr[0] for ctr in sorted(cols_to_remove, key=lambda ctr: -ctr[1]) if ctr[1] > 1))

10000
['Var217', 'Var220', 'Var216', 'Var192', 'Var212']


C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Заменим новые категории (которых очень много) самыми частыми по колонкам

In [52]:
cols_to_drop = []
cleaned_kaggle_data_frame = kaggle_data_frame.copy()
for col in very_best_cat_features:
    column = cleaned_kaggle_data_frame[col]
    train_column = churn_data_frame[col]
    full_train_col = set(train_column.as_matrix())
    kaggle_col = set(column.as_matrix())
    intersected = list(kaggle_col & full_train_col)
    for v in intersected:
        kaggle_col.remove(v)
    frequencies = train_column.value_counts()
    if (len (frequencies) < 1):
        cols_to_drop.append(col)
        continue
    most_frequent_value = frequencies.index[0]
    for to_remove in kaggle_col:
        column = column.replace(to_remove, most_frequent_value)
    cleaned_kaggle_data_frame[col] = column
cols_to_drop

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


[]

Применим теже преобразования, какие мы применяли для обучения тренировочной модели к kaggle датасету.

In [53]:
# very_best_cat_model very_best_cat_features very_best_num_features
kaggle_numeric = cleaned_kaggle_data_frame[very_best_num_features].copy()
kaggle_categorial = cleaned_kaggle_data_frame[very_best_cat_features].copy()

numeric_encoder, categorial_encoder = very_best_cat_model[4]
k_t_n, kaggle_num_means, kaggle_dropped_columns = fill_numericna_means(kaggle_numeric, kaggle_numeric)
kaggle_means_scaled = numeric_encoder.transform(kaggle_num_means)

kaggle_frame_nav = fill_na_frequent_values(kaggle_categorial)
kaggle_encoded = categorial_encoder.transform(kaggle_frame_nav.as_matrix())

X_kaggle = hstack([
    coo_matrix(kaggle_means_scaled),
    kaggle_encoded
])

C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\Pavel\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


Получим вероятности для Kaggle и сохраним их в отдельный файл

In [54]:
kg_predict_probabilities = predict_ridge_proba(X_kaggle, very_best_cat_model[1])
kg_predict_frame = pd.DataFrame(kg_predict_probabilities, columns=["result"])
kg_predict_frame.head()

,result
0,0.410769
1,0.496818
2,0.214427
3,0.448917
4,0.173642


In [55]:
kg_predict_frame.to_csv("churn_kaggle_gb_add_del.csv", index=True, index_label="Id")

## Заключение
В этой модели мы применили
- label encoding и one-hot encoding для обработки категориальных признаков
- scaling для обработки вещественных признаков
- сэмплирование для балансировки классов
- отбор признаков по методу add-del

Нам надо было сделать решение, которое превосходило бы базовое решение на kaggle.
Public: 0.66710

Моя модель на kaggle показала следующие результаты (см. скриншот)
Public: 0.68122

Таким образом можно утверждать, что я улучшил базовое решение.
